In [1]:
import sys
import numpy as np
import MDAnalysis as mda
import matplotlib.pyplot as plt
import os
import fnmatch
import yaml
import gc
import math
import warnings
import re
import time

from optparse import OptionParser
from collections import OrderedDict
from operator import add
from linecache import getline

from datetime import date
today = date.today()

gc.collect()



#class by Joe Melcr
#upgraded by Anne
#from https://github.com/akiirik/Databank/blob/main/Scripts/BuildDatabank/OrderParameter.py


def go_through_simulation(path,name,system,dictonary_of_all_simulations):
    topology = path+name+"/"+name+".gro"
    trajectory = path+name+"/"+name+".xtc"
    top_file = path+name+"/"+name+".top"
    edr_file = path+name+"/"+name+".edr"
    topology_tpr = path+name+"/"+name+".tpr"
    readme = path+name+ "/README.yaml"
    today = str(date.today())
    
    
    dictonary_of_all_simulations[name]={}
    
    u = mda.Universe(topology,trajectory)
    
    if not os.path.isfile(readme):
        sim={}
    else:
        with open(readme) as yaml_file:
            sim = yaml.load(yaml_file, Loader=yaml.FullLoader)
       
    begin_time=u.trajectory.time
    
    timepre_trj=os.path.getmtime(trajectory)
    trj_mod = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre_trj))
    timepre=os.path.getmtime(edr_file)
    edr_mod = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
    
    
    if not 'TRAJECTORY_MODIFIED' in sim or not sim['TRAJECTORY_MODIFIED'] == trj_mod:
        
        
        
        if 'TRAJECTORY_MODIFIED' in sim:
            if not sim['TRAJECTORY_MODIFIED'] == trj_mod:
                sim["MUST_REANALYZE"] = True
        
        sim['TRAJECTORY_MODIFIED'] = trj_mod
        sim['EDR_MODIFIED'] = edr_mod

    


        Nframes=len(u.trajectory)
        timestep = u.trajectory.dt
    
        if not 'TIMESTEP' in sim: 
            sim['TIMESTEP'] = timestep
    
        trj_length = Nframes * timestep
        sim['TRJLENGTH'] = trj_length
        
        sim['TRJBEGIN'] = begin_time
        

        sim['TRJEND'] = begin_time+trj_length
        
        if not 'BINDINGEQ' in sim:
            sim['BINDINGEQ'] = input("Biding of {} eqilibrated after [ps] \n".format(name))
            
            
        if not 'TEMPERATURE' in sim:
            #get temperature from tpr; taken from AddData.py by Anne Kiirikki
            file1 =  'temporary_tpr.txt'

            print("Exporting information with gmx dump")  

            os.system('echo System | gmx dump -s '+ topology_tpr + ' > '+file1)

            with open(file1, 'rt') as tpr_info:
                for line in tpr_info:
                    if 'ref-t' in line:
                        sim['TEMPERATURE']=float(line.split()[1])
  
    
        if not 'COMPOSITION' in sim:
            sim["COMPOSITION"]={}
            with open(top_file,"r") as f:
                molecules_list = False
                for line in f.readlines():
                    if molecules_list:
                        if not line.startswith(";"):
                            items = line.split()
                            if len(items)==2:
                                sim["COMPOSITION"][items[0]]=items[1]
                    elif line.startswith("[ molecules ]"):
                        molecules_list = True

        

    else:

        
        

        if not 'BINDINGEQ' in sim:
            sim['BINDINGEQ'] = input("Biding of {} eqilibrated after [ps] \n".format(output))

        
        if not 'TEMPERATURE' in sim:
            #get temperature from tpr; taken from AddData.py by Anne Kiirikki
            file1 =  'temporary_tpr.txt'

            print("Exporting information with gmx dump")  

            os.system('echo System | gmx dump -s '+ topology_tpr + ' > '+file1)

            with open(file1, 'rt') as tpr_info:
                for line in tpr_info:
                    if 'ref-t' in line:
                        sim['TEMPERATURE']=float(line.split()[1])
  
    
        if not 'COMPOSITION' in sim:
            sim["COMPOSITION"]={}
            with open(top_file,"r") as f:
                molecules_list = False
                for line in f.readlines():
                    if molecules_list:
                        if not line.startswith(";"):
                            items = line.split()
                            if len(items)==2:
                                sim["COMPOSITION"][items[0]]=items[1]
                    elif line.startswith("[ molecules ]"):
                        molecules_list = True

        
    ### TRAJECTORY DICTONARY ###
    #   Contains Size, Modification Date, Timestep, Lenght, Beginning, End 
        
    if not "TRAJECTORY" in sim:
        sim["TRAJECTORY"]={}
        
    sim["TRAJECTORY"]["SIZE"]=os.path.getsize(trajectory)/1048576
    sim['TRAJECTORY']["MODIFIED"] = trj_mod
    
    if 'TRAJECTORY_MODIFIED' in sim:
        del sim['TRAJECTORY_MODIFIED']
        
    if 'TRAJECTORY_SIZE' in sim:   
        del sim['TRAJECTORY_SIZE']
        
        
    Nframes=len(u.trajectory)
    timestep = u.trajectory.dt
    trj_length = Nframes * timestep
        
    sim["TRAJECTORY"]['TIMESTEP'] = timestep
    sim['TRAJECTORY']['LENGTH'] = trj_length
    sim['TRAJECTORY']['BEGIN'] = begin_time
    sim['TRAJECTORY']['END'] = begin_time+trj_length
    
    if 'TIMESTEP' in sim:
        del sim["TIMESTEP"]
        
    
    if 'TRJLENGTH' in sim:
        del sim['TRJLENGTH']
            
        
    if 'TRJBEGIN' in sim:
        del sim['TRJBEGIN']
        
    if 'TRJEND' in sim:
        del sim['TRJEND'] 
        
    ### ENERGY FILE DICTONARY ###
    #   Contains Size, Modification Date 
    
    if not "EDR" in sim:
        sim["EDR"]={}
    
    edr_file = path+name+"/"+name+".edr"
    sim["EDR"]["NAME"] = name+".edr"
    
    try:
        timepre=os.path.getmtime(edr_file)
        edr_mod = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
        sim["EDR"]["SIZE"]=os.path.getsize(edr_file)/1048576
        sim['EDR']["MODIFIED"] = edr_mod
    except:
        sim["EDR"]["SIZE"]= "none"
        sim['EDR']["MODIFIED"] = "none"
        
    
    ### TPR FILE DICTONARY ###
    #   Contains Size, Modification Date 
    
    if not "TPR" in sim:
        sim["TPR"]={}
    
    edr_file = path+name+"/"+name+".tpr"
    sim["TPR"]["NAME"] = name+".tpr"
    
    try:
        timepre=os.path.getmtime(edr_file)
        edr_mod = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
        sim["TPR"]["SIZE"]=os.path.getsize(edr_file)/1048576
        sim['TPR']["MODIFIED"] = edr_mod
    except:
        sim["TPR"]["SIZE"]= "none"
        sim['TPR']["MODIFIED"] = "none"
        
        
    ### TOP FILE DICTONARY ###
    #   Contains Size, Modification Date 
    
    if not "TOP" in sim:
        sim["TOP"]={}
    
    edr_file = path+name+"/"+name+".top"
    sim["TOP"]["NAME"] = name+".top"
    
    try:
        timepre=os.path.getmtime(edr_file)
        edr_mod = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
        sim["TOP"]["SIZE"]=os.path.getsize(edr_file)/1048576
        sim['TOP']["MODIFIED"] = edr_mod
    except:
        sim["TOP"]["SIZE"]= "none"
        sim['TOP']["MODIFIED"] = "none"
    
    
    ### MDP FILE DICTONARY ###
    #   Contains Size, Modification Date 
    
    if not "MDP" in sim:
        sim["MDP"]={}
    
    edr_file = path+name+"/"+name+".mdp"
    sim["MDP"]["NAME"] = name+".mdp"
    
    try:
        timepre=os.path.getmtime(edr_file)
        edr_mod = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
        sim["MDP"]["SIZE"]=os.path.getsize(edr_file)/1048576
        sim['MDP']["MODIFIED"] = edr_mod
    except:
        sim["MDP"]["SIZE"]= "none"
        sim['MDP']["MODIFIED"] = "none"
    
    
    
    ### NDX FILE DICTONARY ###
    #   Contains Size, Modification Date 
    
    if not "NDX" in sim:
        sim["NDX"]={}
    
    edr_file = path+name+"/"+name+".ndx"
    sim["NDX"]["NAME"] = name+".ndx"
    
    try:
        timepre=os.path.getmtime(edr_file)
        edr_mod = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
        sim["NDX"]["SIZE"]=os.path.getsize(edr_file)/1048576
        sim['NDX']["MODIFIED"] = edr_mod
    except:
        sim["NDX"]["SIZE"]= "none"
        sim['NDX']["MODIFIED"] = "none"
        
        
    ### GRO FILE DICTONARY ###
    #   Contains Size, Modification Date 
    
    if not "GRO" in sim:
        sim["GRO"]={}
    
    edr_file = path+name+"/"+name+".gro"
    sim["GRO"]["NAME"] = name+".gro"
    
    try:
        timepre=os.path.getmtime(edr_file)
        edr_mod = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
        sim["GRO"]["SIZE"]=os.path.getsize(edr_file)/1048576
        sim['GRO']["MODIFIED"] = edr_mod
    except:
        sim["GRO"]["SIZE"]= "none"
        sim['GRO']["MODIFIED"] = "none"
    
    
    ### CPT FILE DICTONARY ###
    #   Contains Size, Modification Date 
    
    if not "CPT" in sim:
        sim["CPT"]={}
    
    edr_file = path+name+"/"+name+".cpt"
    sim["CPT"]["NAME"] = name+".cpt"
    
    try:
        timepre=os.path.getmtime(edr_file)
        edr_mod = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
        sim["CPT"]["SIZE"]=os.path.getsize(edr_file)/1048576
        sim['CPT']["MODIFIED"] = edr_mod
    except:
        sim["CPT"]["SIZE"]= "none"
        sim['CPT']["MODIFIED"] = "none"
    
    
    ### LOG FILE DICTONARY ###
    #   Contains Size, Modification Date 
    
    if not "LOG" in sim:
        sim["LOG"]={}
    
    edr_file = path+name+"/"+name+".log"
    sim["LOG"]["NAME"] = name+".log"
    
    try:
        timepre=os.path.getmtime(edr_file)
        edr_mod = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
        sim["LOG"]["SIZE"]=os.path.getsize(edr_file)/1048576
        sim['LOG']["MODIFIED"] = edr_mod
    except:
        sim["LOG"]["SIZE"]= "none"
        sim['LOG']["MODIFIED"] = "none"
    
    
    files_of_interest=["TRAJECTORY","EDR","TPR","TOP","MDP","NDX","GRO","CPT","LOG"]
    for file_name in files_of_interest:
        dictonary_of_all_simulations[name][file_name]={}
        dictonary_of_all_simulations[name][file_name]["SIZE"]=sim[file_name]["SIZE"]
        dictonary_of_all_simulations[name][file_name]["MODIFIED"]=sim[file_name]["MODIFIED"]
    
    try:
        
        folder2020="/media/nenciric/Ricky2020/simulations/"+name+"/"
        folder2020_1="/media/nenciric/Ricky20201/2020/simulations/"+name+"/"
        
        if os.path.isdir(folder2020):
            print("\n \n Files in 2020 \n")
            for file in os.listdir(folder2020):
                input_corr_file = folder2020+os.fsdecode(file)
                file_size=os.path.getsize(input_corr_file)/1048576
                timepre=os.path.getmtime(input_corr_file)
                file_modified = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
                print("{}, size: {} Mb, {} \n".format(os.fsdecode(file),file_size,file_modified))
                
        if os.path.isdir(folder2020_1):
            print("\n \n Files in 2020_1 \n")
            for file in os.listdir(folder2020_1):
                input_corr_file = folder2020_1+os.fsdecode(file)
                file_size=os.path.getsize(input_corr_file)/1048576
                timepre=os.path.getmtime(input_corr_file)
                file_modified = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
                print("{}, size: {} Mb, {} \n".format(os.fsdecode(file),file_size,file_modified))
    except Exception as e:
        print(e)
        
        
        
    try:
        trajectory2020="/media/nenciric/Ricky2020/simulations/"+name+"/"+name+".xtc"
        trajectory2020_1="/media/nenciric/Ricky20201/2020/simulations/"+name+"/"+name+".xtc"
        
        sim["DELETE"]=False
        timepre=os.path.getmtime(trajectory2020)
        time_to_print1 = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
        timepre=os.path.getmtime(trajectory2020_1)
        time_to_print2 = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
        size_to_print1 = os.path.getsize(trajectory2020)/1048576
        size_to_print2 = os.path.getsize(trajectory2020_1)/1048576
        if size_to_print1 == sim["TRAJECTORY_SIZE"] or size_to_print2 == sim["TRAJECTORY_SIZE"]:
            sim["DELETE"]=True
        else:
            timepre=os.path.getmtime(trajectory2020)
            time_to_print1 = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
            timepre=os.path.getmtime(trajectory2020_1)
            time_to_print2 = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
            if os.path.isfile(trajectory2020) and os.path.isfile(trajectory2020_1):
                print("One Touch: {}, 2020:  {}, 2020_1 {}".format(sim["TRAJECTORY_MODIFIED"],time_to_print1,time_to_print2))
            elif not os.path.isfile(trajectory2020):
                print("Traj 2020 does not exist")
            elif not os.path.isfile(trajectory2020_1):
                print("Traj 2020_1 does not exist")
                
                
        if os.path.getmtime(trajectory2020_1) == timepre_trj:
            sim["DELETE"]=True
    except:
        sim["DELETE"]=False
        
        if os.path.isfile(trajectory2020) and os.path.isfile(trajectory2020_1):
            timepre=os.path.getmtime(trajectory2020)
            time_to_print1 = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
            timepre=os.path.getmtime(trajectory2020_1)
            time_to_print2 = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timepre))
            print("One Touch: {}, 2020:  {}, 2020_1 {}".format(sim["TRAJECTORY_MODIFIED"],time_to_print1,time_to_print2))
        elif not os.path.isfile(trajectory2020):
            print("Traj 2020 does not exist")
        elif not os.path.isfile(trajectory2020_1):
            print("Traj 2020_1 does not exist")
            
    
    with open(readme, 'w') as f:
        yaml.dump(sim,f, sort_keys=False)
    
    with open(system +"_"+ today+ '.out', 'a') as f:
        try:
            f.write("{:80} {:>5} {:>6} {:>4} {:>4} {:>4} ".format(name,int(timestep),sim["TRAJECTORY"]['LENGTH']/1000,int(int(sim['BINDINGEQ'])/1000),sim['TEMPERATURE'],sim['DELETE']))
        except:
            f.write("{:80} {:>5} {:>6} {:>4} {:>4} {:>4} ".format(name,int(timestep),sim["TRAJECTORY"]['LENGTH']/1000,sim['BINDINGEQ'],sim['TEMPERATURE'],sim['DELETE']))

        f.write("\n")
        
    print("great success!!!")
    
def initialize_output(system,folder_path):
    today = str(date.today())
    with open(system +"_"+ today+ '.out', 'w') as f:
        f.write("# List of simulations for: {} \n".format(system))
        f.write("# Path: {} \n".format(folder_path))
        f.write("# Create on: {} \n \n".format(today))
        f.write("#     1 - System \n")
        f.write("#     2 - Saving frequency [ps]  \n")
        f.write("#     3 - Total length [ns] \n")
        f.write("#     4 - Binding equilibrated after [ns]  \n")
        f.write("#     5 - Temperature [K] \n")
        f.write("#     6 - Composition \n \n")







In [2]:
"""Go through all simulations and calculate OP and box dimentions"""
folder_path="2020/simulations/"
systems=["all"]

dictonary_of_all_simulations={}

for file in os.listdir(folder_path):
    for system in systems:
        initialize_output(system,folder_path)


for file in os.listdir(folder_path):
    input_corr_file = folder_path+os.fsdecode(file)
    for system in systems:
        go_through_simulation(folder_path,os.fsdecode(file),system,dictonary_of_all_simulations)

with open("all_simulations_Ricky2020_samulis.yaml", 'w') as f:
    yaml.dump(dictonary_of_all_simulations,f, sort_keys=True)

Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great suc

/home/ricky/anaconda3/envs/all3/lib/python3.6/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: M
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great success!!!
Traj 2020 does not exist
great suc

In [6]:
folder_path="2020/simulations/"
systems=["etidocaine","dibucaine","SMS","TPP"]




for file in os.listdir(folder_path):
    input_corr_file = folder_path+os.fsdecode(file)
    for system in systems:
        if fnmatch.fnmatch(os.fsdecode(file), "*"+system+"*"):
            print(os.fsdecode(file))
            readme=input_corr_file+"/README.yaml"
            
            try:
                with open(readme) as yaml_file:
                    sim = yaml.load(yaml_file, Loader=yaml.FullLoader)
            except:
                sim = {}
                sim['TPR']={}
            
            topology_tpr=folder_path+os.fsdecode(file)+'/'+os.fsdecode(file)+'.tpr'
            
            try:
                with open(topology_tpr, 'rb') as tpr_info:
                    for line in tpr_info:
                        if 'VERSION' in str(line):
                            sim['TPR']['GMX_VERSION']=(str(line.split()[1])[2:8])
                            print(sim['TPR']['GMX_VERSION'])
            except:
                print("fail")
            
            with open(readme, 'w') as f:
                yaml.dump(sim,f, sort_keys=False)

dibucaine_POPC_CHARMM_298K_Cl_countra_104mM_8800waters
2018.6
dibucaine_POPC_CHARMM_298K_Cl_countra_123mM
2018.6
dibucaine_POPC_CHARMM_298K_Cl_countra_130mM_8800waters
2018.6
dibucaine_POPC_CHARMM_298K_Cl_countra_13mM_8800waters
2018.6
dibucaine_POPC_CHARMM_298K_Cl_countra_156mM_8800waters
2018.6
dibucaine_POPC_CHARMM_298K_Cl_countra_15mM_177600waters
2019.3
dibucaine_POPC_CHARMM_298K_Cl_countra_182mM_8800waters
2018.6
dibucaine_POPC_CHARMM_298K_Cl_countra_208mM_8800waters
2018.6
dibucaine_POPC_CHARMM_298K_Cl_countra_234mM_8800waters
2018.6
dibucaine_POPC_CHARMM_298K_Cl_countra_260mM_8800waters
2018.6
etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_7581waters
2018.6
etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_9975waters
2018.6
etidocaine_POPC_CHARMM_298K_Cl_countra_280mM_19900waters
2018.6
etidocaine_POPC_CHARMM_298K_Cl_countra_280mM_19900waters_paramchem
2018.6
etidocaine_POPC_CHARMM_298K_Cl_countra_280mM_5200waters
2018.6
etidocaine_POPC_CHARMM_298K_Cl_countra_280mM_5200waters_paramche